In [1]:
from safetensors import safe_open


with safe_open("/models/train/vllm-weights/v2/li-EAGLE-LLaMA3-Instruct-70B/model.safetensors", 'pt', device='cpu') as f:
	for key in f.keys():
		print(key, f.get_tensor(key).shape)


lm_head.weight torch.Size([128256, 8192])
model.embed_tokens.weight torch.Size([128256, 8192])
model.fc.weight torch.Size([8192, 16384])
model.layers.0.mlp.down_proj.weight torch.Size([8192, 28672])
model.layers.0.mlp.gate_up_proj.weight torch.Size([57344, 8192])
model.layers.0.post_attention_layernorm.weight torch.Size([8192])
model.layers.0.self_attn.o_proj.weight torch.Size([8192, 8192])
model.layers.0.self_attn.qkv_proj.weight torch.Size([10240, 8192])


In [2]:
from safetensors import safe_open


with safe_open("/models/train/vllm-weights/v1/20240910-eagle-llama3.1-8b/new.safetensors", 'pt', device='cpu') as f:
	for key in f.keys():
		print(key, f.get_tensor(key).shape)


lm_head.weight torch.Size([128256, 4096])
model.embed_tokens.weight torch.Size([128256, 4096])
model.fc.bias torch.Size([4096])
model.fc.weight torch.Size([4096, 8192])
model.layers.0.mlp.down_proj.weight torch.Size([4096, 14336])
model.layers.0.mlp.gate_up_proj.weight torch.Size([28672, 4096])
model.layers.0.post_attention_layernorm.weight torch.Size([4096])
model.layers.0.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.qkv_proj.weight torch.Size([6144, 4096])


In [3]:
import torch


indices = torch.load('/root/work/chat-regen/tools/frequent-tokens.pt')
indices.shape, indices

tail_indices = torch.arange(256).long() + 128000
new_indices = torch.cat([indices[:32000], tail_indices], dim=0)
new_indices.shape

torch.Size([32256])

In [4]:
new_indices

tensor([    11,    323,    279,  ..., 128253, 128254, 128255])

In [7]:
state = dict()

with safe_open("/models/train/vllm-weights/v1/20240910-eagle-llama3.1-8b/new.safetensors", 'pt', device='cpu') as f:
	for key in f.keys():
		w = f.get_tensor(key)
		if key in ['lm_head.weight', 'model.embed_tokens.weight']:
			state[key] = w[new_indices]
		else:
			state[key] = w

	print(f.metadata())

state

None


{'lm_head.weight': tensor([[ 0.0146,  0.0142, -0.0009,  ..., -0.0039, -0.0240, -0.0173],
         [-0.0018,  0.0020,  0.0089,  ...,  0.0028, -0.0125, -0.0164],
         [-0.0093,  0.0119, -0.0164,  ...,  0.0247, -0.0054, -0.0065],
         ...,
         [-0.0038,  0.0020,  0.0034,  ...,  0.0003,  0.0081,  0.0083],
         [-0.0038,  0.0020,  0.0034,  ...,  0.0003,  0.0081,  0.0083],
         [-0.0038,  0.0020,  0.0034,  ...,  0.0003,  0.0081,  0.0083]]),
 'model.embed_tokens.weight': tensor([[-2.5558e-04, -4.1199e-04,  3.6478e-05,  ...,  2.1267e-04,
           3.1471e-05,  2.9182e-04],
         [-6.4087e-04, -2.9297e-03,  1.1673e-03,  ...,  6.7234e-05,
           1.3962e-03,  5.1880e-03],
         [ 5.0354e-04, -1.9989e-03, -1.3504e-03,  ...,  1.4572e-03,
          -5.4321e-03,  3.0518e-03],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          -0.0000e+00, -0.0000e+00],
         [ 0.0000e+00, -0.0000e+00, -0.0000e+00,  ...,  0.0000e+00,
         

In [8]:
from safetensors.torch import save_file


save_file(state, "/models/train/vllm-weights/v2/20240910-eagle-llama3.1-8b-vocab32k/model.safetensors", metadata={'format': 'pt'})